# Analysis

After generating the models for all the algorithmns for different runtimes, csv files are generated for each model with the hyperparameters values. These hyperparameters are analysed to find three things:

1. Trying to find the important hyperparameters
2. Trying to find the range of hyperparameters
3. Compare the range of values across the models for different hyperparameters

In [233]:
#importing h2o libraries
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil

import warnings
warnings.filterwarnings('ignore')



In [234]:
# importing libraries
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import re

In [235]:
#starting h2o 
port_no=random.randint(5555,55555)
h2o.init(strict_version_check=False,min_mem_size_GB=5,port=port_no)

Checking whether there is an H2O instance running at http://localhost:51199..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_101"; Java(TM) SE Runtime Environment (build 1.8.0_101-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.101-b13, mixed mode)
  Starting server from /Applications/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/gf/p944hdxd7jq73_9cxfglxn540000gn/T/tmpnavlvu22
  JVM stdout: /var/folders/gf/p944hdxd7jq73_9cxfglxn540000gn/T/tmpnavlvu22/h2o_vaish_started_from_python.out
  JVM stderr: /var/folders/gf/p944hdxd7jq73_9cxfglxn540000gn/T/tmpnavlvu22/h2o_vaish_started_from_python.err
  Server is running at http://127.0.0.1:51199
Connecting to H2O server at http://127.0.0.1:51199... successful.


H2O cluster uptime:,03 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.6
H2O cluster version age:,1 month and 12 days
H2O cluster name:,H2O_from_python_vaish_19v214
H2O cluster total nodes:,1
H2O cluster free memory:,4.792 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


Data set is imported using pandas as it is needed for GridSearch to find important hyperparameters

In [236]:
dfd=pd.read_csv("anime_filtered.csv", decimal = ',')

In [237]:
dfd.dtypes

anime_id           int64
title             object
title_english     object
title_japanese    object
title_synonyms    object
image_url         object
type              object
source            object
episodes           int64
status            object
airing              bool
aired_string      object
aired             object
duration          object
rating            object
score             object
scored_by          int64
rank              object
popularity         int64
members            int64
favorites          int64
background        object
premiered         object
broadcast         object
related           object
producer          object
licensor          object
studio            object
genre             object
opening_theme     object
ending_theme      object
dtype: object

As the datatypes of some real values are object, we have to convert all of them to float

In [238]:
dfd['score'] = dfd.score.astype(float)

In [239]:
ser = pd.Series(dfd['rank'])
dfd['rank'] = ser.astype('float')

Categorical columns and boolean values are converted to integers and 

In [240]:
dfd['status'] = dfd['status'].astype('category').cat.codes
dfd['source'] = dfd['source'].astype('category').cat.codes
dfd['type'] = dfd['type'].astype('category').cat.codes

In [241]:
dfd = dfd.applymap(lambda x: 1 if x == True else x)
dfd = dfd.applymap(lambda x: 0 if x == False else x)

In [242]:
dfd.dtypes

anime_id            int64
title              object
title_english      object
title_japanese     object
title_synonyms     object
image_url          object
type                int64
source              int64
episodes            int64
status              int64
airing              int64
aired_string       object
aired              object
duration           object
rating             object
score             float64
scored_by           int64
rank              float64
popularity          int64
members             int64
favorites           int64
background         object
premiered          object
broadcast          object
related            object
producer           object
licensor           object
studio             object
genre              object
opening_theme      object
ending_theme       object
dtype: object

X and Y are initialised with dataset columns with score as target. X is initialized with columns whose datatypes are int, float , real values

In [243]:
columns = ['type','source','episodes','status','airing','scored_by','popularity','members','favorites']
X = dfd[columns]
y = dfd.score

__XGBoost__ 

In [84]:
#importing the file
# df=pd.read_csv("XGboost.csv", decimal = ',')
df = h2o.import_file(path="XGboost.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [85]:
df.head()

name,nfolds,seed,solver,kc_val_mo,kc_val_pr,fold_assignment,ntrees,max_runtime_secs,stopping_rounds,stopping_metric,stopping_tolerance,max_depth,learn_rate,min_rows,sample_rate,col_sam_rate,col_sam_r_per_tree,score_tree_interval
XGBoost_1_AutoML_20190418_132524,5,5.9207e+18,nan,False,True,Modulo,197,8,3,deviance,0.008312,20,0.05,10,0.6,0.8,0.8,5
XGBoost_1_AutoML_20190418_132524,5,5.9207e+18,nan,False,True,Modulo,197,8,3,deviance,0.008312,20,0.05,10,0.6,0.8,0.8,5
XGBoost_1_AutoML_20190418_132524,5,5.9207e+18,nan,False,True,Modulo,197,8,3,deviance,0.008312,20,0.05,10,0.6,0.8,0.8,5
XGBoost_1_AutoML_20190418_132524,5,5.9207e+18,nan,False,True,Modulo,197,8,3,deviance,0.008312,20,0.05,10,0.6,0.8,0.8,5
XGBoost_1_AutoML_20190418_132524,5,5.9207e+18,nan,False,True,Modulo,197,8,3,deviance,0.008312,20,0.05,10,0.6,0.8,0.8,5
XGBoost_1_AutoML_20190418_132524,5,5.9207e+18,nan,False,True,Modulo,197,8,3,deviance,0.008312,20,0.05,10,0.6,0.8,0.8,5
XGBoost_1_AutoML_20190418_135050,5,7.23078e+18,nan,False,True,Modulo,127,53,3,deviance,0.008312,20,0.05,10,0.6,0.8,0.8,5
XGBoost_1_AutoML_20190418_135050,5,7.23078e+18,nan,False,True,Modulo,127,53,3,deviance,0.008312,20,0.05,10,0.6,0.8,0.8,5
XGBoost_1_AutoML_20190418_135050,5,7.23078e+18,nan,False,True,Modulo,127,53,3,deviance,0.008312,20,0.05,10,0.6,0.8,0.8,5
XGBoost_1_AutoML_20190418_135050,5,7.23078e+18,nan,False,True,Modulo,127,53,3,deviance,0.008312,20,0.05,10,0.6,0.8,0.8,5


To pass the parameters in grid search, we need all the values in a list. Hence all the unique values of ntrees,minrows,col_sam_r_per_tree are taken and converted to list.

In [86]:
pd = df.as_data_frame()
ntrees = pd['ntrees'].unique().tolist()

minrows = pd['min_rows'].unique().tolist()
col_sam_r_per_tree = pd['col_sam_r_per_tree'].unique().tolist()

In [88]:
df.types

{'name': 'enum',
 'nfolds': 'int',
 'seed': 'int',
 'solver': 'int',
 'kc_val_mo': 'enum',
 'kc_val_pr': 'enum',
 'fold_assignment': 'enum',
 'ntrees': 'int',
 'max_runtime_secs': 'int',
 'stopping_rounds': 'int',
 'stopping_metric': 'enum',
 'stopping_tolerance': 'real',
 'max_depth': 'int',
 'learn_rate': 'real',
 'min_rows': 'real',
 'sample_rate': 'real',
 'col_sam_rate': 'real',
 'col_sam_r_per_tree': 'real',
 'score_tree_interval': 'int'}

Here, we are finding the range of all the column which are numerical and added to a list called ranges.

In [90]:
ranges = []
for key,val in df.types.items():
    print(key,val)
    if val == 'int': 
         ranges.append([key,[df[key].min(),df[key].max()]])
    if val == 'real':
         ranges.append([key,[df[key].min(),df[key].max()]])

name enum
nfolds int
seed int
solver int
kc_val_mo enum
kc_val_pr enum
fold_assignment enum
ntrees int
max_runtime_secs int
stopping_rounds int
stopping_metric enum
stopping_tolerance real
max_depth int
learn_rate real
min_rows real
sample_rate real
col_sam_rate real
col_sam_r_per_tree real
score_tree_interval int


__2 Trying to find the range of hyperparameters__

In [91]:
ranges

[['nfolds', [5.0, 5.0]],
 ['seed', [-8.186597997308668e+18, 8.50284134061186e+18]],
 ['solver', [nan, nan]],
 ['ntrees', [13.0, 276.0]],
 ['max_runtime_secs', [0.0, 53.0]],
 ['stopping_rounds', [0.0, 3.0]],
 ['stopping_tolerance', [0.008312003470029437, 0.008312003470029437]],
 ['max_depth', [5.0, 20.0]],
 ['learn_rate', [0.05, 0.05]],
 ['min_rows', [0.01, 20.0]],
 ['sample_rate', [0.6, 1.0]],
 ['col_sam_rate', [0.6, 1.0]],
 ['col_sam_r_per_tree', [0.7, 1.0]],
 ['score_tree_interval', [5.0, 5.0]]]

__1 Trying to find the important hyperparameters__

In Grid Search CV , XGBRegressor is taken and the parameters which can be passed in GridSearch are initialised and the model is fitted with X and y.

In [140]:

from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
# Create the parameter grid based on the results of random search 
param_grid = {
    'ntrees': ntrees,
    'stopping_rounds': [0, 3],
    'max_depth' : [5,10,15 ,20],
    'learn_rate': [0.05],
    'min_rows': minrows,
    'sample_rate': [0.6,0.8,1],
    'col_sample_rate': [0.6, 1.0],
    'col_sample_rate_per_tree' : col_sam_r_per_tree

}
# Create a based model
clf = XGBRegressor()
# Instantiate the grid search model
xgb = GridSearchCV(estimator = clf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [141]:
re = xgb.fit(X,y)

Fitting 3 folds for each of 24192 candidates, totalling 72576 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed: 15.4min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed: 16.6min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed: 17.8min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 19.4min
[Parallel(n_jobs=-1)]: Done 3265 tasks      | elapsed: 21.1min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 23.1min
[Parallel(n_jobs=-1)]: Done 4885 tasks      | elapsed: 25.3min
[Parallel(n_jobs=-1)]: Done 5816 tasks      | elapsed: 27.6min
[Parallel(n_jobs=-1)]: Done 6829 tasks      | elapsed: 30.7min
[Parallel(n_jobs=-1)]: Done 7922 tasks      | elapsed: 33.4min
[Parallel(n_jobs=-1)]: Done 9097 tasks      | e

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'ntrees': [197, 127, 119, 120, 121, 152, 13, 203, 110, 20, 145, 276, 79, 60, 161, 172, 98, 92], 'stopping_rounds': [0, 3], 'max_depth': [5, 10, 15, 20], 'learn_rate': [0.05], 'min_rows': [10.0, 20.0, 1.0, 0.1, 5.0, 3.0, 0.01], 'sample_rate': [0.6, 0.8, 1], 'col_sample_rate': [0.6, 1.0], 'col_sample_rate_per_tree': [0.8, 0.9, 1.0, 0.7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

These are the best parameters for XGB given by GridSearch 

In [142]:
xgb.best_params_

{'col_sample_rate': 0.6,
 'col_sample_rate_per_tree': 0.8,
 'learn_rate': 0.05,
 'max_depth': 5,
 'min_rows': 10.0,
 'ntrees': 197,
 'sample_rate': 0.6,
 'stopping_rounds': 0}

__GBM__

GBM hyperparameters are imported for the analysis 

In [268]:
df_anime = h2o.import_file(path="anime_filtered.csv")
x1 = [name for name in df_anime.columns if name != 'score'] 
y1 ='score'

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [269]:
#importing the file
df = h2o.import_file(path="GBM.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [270]:
df.head(40)

gbm_id,name,nfolds,max_depth,min_rows,distribution,stopping_metric,stopping_tolerance,seed,sample_rate
1,GBM_grid_1_AutoML_20190418_142243_model_3,5,6,6,gaussian,deviance,0.008312,-7.92e+18,0.6
2,GBM_1_AutoML_20190418_144229,5,15,15,gaussian,deviance,0.008312,3.08e+18,0.8
3,GBM_grid_1_AutoML_20190418_142243_model_2,5,10,10,gaussian,deviance,0.008312,3.07e+18,0.7
4,GBM_grid_1_AutoML_20190418_140346_model_3,5,4,4,gaussian,deviance,0.008312,-5.29e+18,0.5
5,GBM_3_AutoML_20190418_140346,5,15,15,gaussian,deviance,0.008312,-7.17e+18,0.8
6,GBM_5_AutoML_20190418_150256,5,15,15,gaussian,deviance,0.008312,-5.05e+18,0.8
7,GBM_grid_1_AutoML_20190418_144229_model_4,5,12,12,gaussian,deviance,0.008312,3.42e+18,0.6
8,GBM_2_AutoML_20190418_140346,5,15,15,gaussian,deviance,0.008312,-7.17e+18,0.8
9,GBM_2_AutoML_20190418_144229,5,15,15,gaussian,deviance,0.008312,3.08e+18,0.8
10,GBM_4_AutoML_20190418_142243,5,15,15,gaussian,deviance,0.008312,3.67e+18,0.8


Some columns are converted to list for Grid Search 

In [271]:
pd = df.as_data_frame()
minrows = pd['min_rows'].unique().tolist()
maxdepth = pd['max_depth'].unique().tolist()
sample_rate = pd['sample_rate'].unique().tolist()

In [272]:
maxdepth

[6, 15, 10, 4, 12, 7, 3, 8, 5, 16, 13, 11]

__2 Trying to find the range of hyperparameters__

In [273]:
df.types

{'gbm_id': 'int',
 'name': 'string',
 'nfolds': 'int',
 'max_depth': 'int',
 'min_rows': 'int',
 'distribution': 'enum',
 'stopping_metric': 'enum',
 'stopping_tolerance': 'real',
 'seed': 'int',
 'sample_rate': 'real'}

Here, we are finding the range of all the column which are numerical and added to a list called ranges.

In [274]:
ranges = []
for key,val in df.types.items():
    print(key,val)
    if val == 'int': 
         ranges.append([key,[df[key].min(),df[key].max()]])
    if val == 'real':
         ranges.append([key,[df[key].min(),df[key].max()]])

gbm_id int
name string
nfolds int
max_depth int
min_rows int
distribution enum
stopping_metric enum
stopping_tolerance real
seed int
sample_rate real


Here are the ranges for GBM hyperparameters.

In [275]:
ranges

[['gbm_id', [1.0, 39.0]],
 ['nfolds', [5.0, 5.0]],
 ['max_depth', [3.0, 16.0]],
 ['min_rows', [3.0, 16.0]],
 ['stopping_tolerance', [0.008312003, 0.008312003]],
 ['seed', [-9e+18, 8.93e+18]],
 ['sample_rate', [0.5, 1.0]]]

__1. Trying to find the important hyperparameters__

In Grid Search, only max_depth is passed and fit the model to find rmse and similarly all the other hyperparameters are passed and compared the rmse to find which hyperparameters are important

In [276]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch

In [278]:
gbm_params1 = {
                'max_depth': [6, 15, 10, 4, 12, 7, 3, 8, 5, 16, 13, 11]}

# Train and validate a cartesian grid of GBMs
gbm_grid1 = H2OGridSearch(model=H2OGradientBoostingEstimator,
                          grid_id='gbm_grid1',
                          hyper_params=gbm_params1)
gbm_grid1.train(x=x, y=y,
                training_frame=df_anime)

In [279]:
# Get the grid results, sorted by validation AUC
gbm_gridperf1 = gbm_grid1.get_grid(sort_by='rmse', decreasing=False)
gbm_gridperf1

     max_depth           model_ids                 rmse
0           16  gbm_grid1_model_10  0.08933873399569765
1           15   gbm_grid1_model_2   0.0998722504624683
2           13  gbm_grid1_model_11   0.1210972373228109
3           12   gbm_grid1_model_5  0.13222554212587315
4           11  gbm_grid1_model_12  0.14707683264903976
5           10   gbm_grid1_model_3  0.16140776961456432
6            8   gbm_grid1_model_8   0.2128587745737397
7            7   gbm_grid1_model_6  0.24957342103409744
8            6   gbm_grid1_model_1   0.2868765688648202
9            5   gbm_grid1_model_9   0.3365502135360817
10           4   gbm_grid1_model_4  0.39514456288274463
11           5  gbm_grid1_model_25   0.3992161522478504
12           5  gbm_grid1_model_26  0.40204459040039614
13           5  gbm_grid1_model_22  0.40482131597778365
14           5  gbm_grid1_model_27  0.40544190037804073
15           5  gbm_grid1_model_19  0.40609216328586173
16           5  gbm_grid1_model_24   0.407693415

In [262]:
gbm_params1 = {
                'sample_rate': sample_rate}

# Train and validate a cartesian grid of GBMs
gbm_grid1 = H2OGridSearch(model=H2OGradientBoostingEstimator,
                          grid_id='gbm_grid1',
                          hyper_params=gbm_params1)
gbm_grid1.train(x=x, y=y,
                training_frame=df_anime)

In [263]:
gbm_gridperf2 = gbm_grid1.get_grid(sort_by='rmse', decreasing=False)
gbm_gridperf2

     max_depth           model_ids                 rmse
0           16  gbm_grid1_model_10  0.08933873399569765
1           15   gbm_grid1_model_2   0.0998722504624683
2           13  gbm_grid1_model_11   0.1210972373228109
3           12   gbm_grid1_model_5  0.13222554212587315
4           11  gbm_grid1_model_12  0.14707683264903976
5           10   gbm_grid1_model_3  0.16140776961456432
6            8   gbm_grid1_model_8   0.2128587745737397
7            7   gbm_grid1_model_6  0.24957342103409744
8            6   gbm_grid1_model_1   0.2868765688648202
9            5   gbm_grid1_model_9   0.3365502135360817
10           4   gbm_grid1_model_4  0.39514456288274463
11           5  gbm_grid1_model_18   0.4134265528913023
12           5  gbm_grid1_model_17   0.4187941711188329
13           5  gbm_grid1_model_14   0.4246893756201058
14           5  gbm_grid1_model_15  0.43410397517723565
15           5  gbm_grid1_model_13  0.44266815339118665
16           3   gbm_grid1_model_7   0.443862664

In [265]:
gbm_params1 = {
                'min_rows': minrows}

# Train and validate a cartesian grid of GBMs
gbm_grid1 = H2OGridSearch(model=H2OGradientBoostingEstimator,
                          grid_id='gbm_grid1',
                          hyper_params=gbm_params1)
gbm_grid1.train(x=x, y=y,
                training_frame=df_anime)

In [266]:
gbm_gridperf3 = gbm_grid1.get_grid(sort_by='rmse', decreasing=False)
gbm_gridperf3

     max_depth           model_ids                 rmse
0           16  gbm_grid1_model_10  0.08933873399569765
1           15   gbm_grid1_model_2   0.0998722504624683
2           13  gbm_grid1_model_11   0.1210972373228109
3           12   gbm_grid1_model_5  0.13222554212587315
4           11  gbm_grid1_model_12  0.14707683264903976
5           10   gbm_grid1_model_3  0.16140776961456432
6            8   gbm_grid1_model_8   0.2128587745737397
7            7   gbm_grid1_model_6  0.24957342103409744
8            6   gbm_grid1_model_1   0.2868765688648202
9            5   gbm_grid1_model_9   0.3365502135360817
10           4   gbm_grid1_model_4  0.39514456288274463
11           5  gbm_grid1_model_25   0.3992161522478504
12           5  gbm_grid1_model_26  0.40204459040039614
13           5  gbm_grid1_model_22  0.40482131597778365
14           5  gbm_grid1_model_27  0.40544190037804073
15           5  gbm_grid1_model_19  0.40609216328586173
16           5  gbm_grid1_model_24   0.407693415

__GLM__

In [201]:
df_anime = h2o.import_file(path="anime_filtered.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [205]:
X =df_anime
def check_X(x,df):
    for name in x:
        if name not in df.columns:
            x.remove(name)  
    return x    

X=check_X(X,df_anime)

GLM hyperparameters file is imported for analysis.

In [206]:
#importing the file
df = h2o.import_file(path="GLM.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [207]:
df.head()

glm_id,name,nfolds,seed,solver,kc_val_mo,kc_val_pr,fold_assignment,lambda_search,max_iterations,objective_epsilon,gradient_epsilon,link,lambda_min_ratio,obj_reg
1,GLM_grid_1_AutoML_20190418_132524_model_1,5,3.52e+18,L_BFGS,FALSE,TRUE,Modulo,TRUE,300,300,1e-06,identity,0.0001,6.91e-05
2,GLM_grid_1_AutoML_20190418_135050_model_1,5,-7.58e+16,L_BFGS,FALSE,TRUE,Modulo,TRUE,300,300,1e-06,identity,0.0001,6.91e-05
3,GLM_grid_1_AutoML_20190418_140346_model_1,5,6.09e+18,L_BFGS,FALSE,TRUE,Modulo,TRUE,300,300,1e-06,identity,0.0001,6.91e-05
4,GLM_grid_1_AutoML_20190418_142243_model_1,5,4.02e+18,L_BFGS,FALSE,TRUE,Modulo,TRUE,300,300,1e-06,identity,0.0001,6.91e-05
5,GLM_grid_1_AutoML_20190418_144229_model_1,5,6.33e+18,L_BFGS,FALSE,TRUE,Modulo,TRUE,300,300,1e-06,identity,0.0001,6.91e-05
6,GLM_grid_1_AutoML_20190418_150256_model_1,5,-5.91e+18,L_BFGS,FALSE,TRUE,Modulo,TRUE,300,300,1e-06,identity,0.0001,6.91e-05


__2 Trying to find the range of hyperparameters__

In [208]:
df.types

{'glm_id': 'int',
 'name': 'string',
 'nfolds': 'int',
 'seed': 'int',
 'solver': 'enum',
 'kc_val_mo': 'enum',
 'kc_val_pr': 'enum',
 'fold_assignment': 'enum',
 'lambda_search': 'enum',
 'max_iterations': 'int',
 'objective_epsilon': 'int',
 'gradient_epsilon': 'real',
 'link': 'enum',
 'lambda_min_ratio': 'real',
 'obj_reg': 'real'}

Here, we are finding the range of all the column which are numerical and added to a list called ranges.

In [209]:
ranges = []
for key,val in df.types.items():
    print(key,val)
    if val == 'int': 
         ranges.append([key,[df[key].min(),df[key].max()]])
    if val == 'real':
         ranges.append([key,[df[key].min(),df[key].max()]])

glm_id int
name string
nfolds int
seed int
solver enum
kc_val_mo enum
kc_val_pr enum
fold_assignment enum
lambda_search enum
max_iterations int
objective_epsilon int
gradient_epsilon real
link enum
lambda_min_ratio real
obj_reg real


In [210]:
ranges

[['glm_id', [1.0, 6.0]],
 ['nfolds', [5.0, 5.0]],
 ['seed', [-5.91e+18, 6.33e+18]],
 ['max_iterations', [300.0, 300.0]],
 ['objective_epsilon', [300.0, 300.0]],
 ['gradient_epsilon', [1e-06, 1e-06]],
 ['lambda_min_ratio', [0.0001, 0.0001]],
 ['obj_reg', [6.91e-05, 6.91e-05]]]

In GLM Grid Search , the only parameters that are accepted are seed from the above list. 

__XRT__

Importing XRT hyperparameter file for the analysis.

In [186]:
#importing the file
df = h2o.import_file(path="XRT.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [187]:
df.head()

xrt_id,name,nfolds,seed,kc_val_mo,kc_val_pr,fold_assignment,ntrees,distributionstopping_metric,stopping_tolerance,histogram_type
1,XRT_1_AutoML_20190418_132524,5,-6.68e+18,FALSE,TRUE,Modulo,32,nan,0.008312,Random
2,XRT_1_AutoML_20190418_135050,5,6.82e+18,FALSE,TRUE,Modulo,46,nan,0.008312,Random
3,XRT_1_AutoML_20190418_140346,5,6.72e+18,FALSE,TRUE,Modulo,14,nan,0.008312,Random
4,XRT_1_AutoML_20190418_142243,5,2.63e+18,FALSE,TRUE,Modulo,18,nan,0.008312,Random
5,XRT_1_AutoML_20190418_144229,5,2.04e+18,FALSE,TRUE,Modulo,50,nan,0.008312,Random
6,XRT_1_AutoML_20190418_150256,5,-7.04e+18,FALSE,TRUE,Modulo,50,nan,0.008312,Random


Stored the values of ntrees values in list for Grid Search

In [195]:
pd = df.as_data_frame()
ntrees = pd['ntrees'].unique().tolist()


[32, 46, 14, 18, 50]

__2 Trying to find the range of hyperparameters__

In [189]:
df.types

{'xrt_id': 'int',
 'name': 'string',
 'nfolds': 'int',
 'seed': 'int',
 'kc_val_mo': 'enum',
 'kc_val_pr': 'enum',
 'fold_assignment': 'enum',
 'ntrees': 'int',
 'distributionstopping_metric': 'int',
 'stopping_tolerance': 'real',
 'histogram_type': 'enum'}

Here, we are finding the range of all the column which are numerical and added to a list called ranges.

In [190]:
ranges = []
for key,val in df.types.items():
    print(key,val)
    if val == 'int': 
         ranges.append([key,[df[key].min(),df[key].max()]])
    if val == 'real':
         ranges.append([key,[df[key].min(),df[key].max()]])

xrt_id int
name string
nfolds int
seed int
kc_val_mo enum
kc_val_pr enum
fold_assignment enum
ntrees int
distributionstopping_metric int
stopping_tolerance real
histogram_type enum


In [191]:
ranges

[['xrt_id', [1.0, 6.0]],
 ['nfolds', [5.0, 5.0]],
 ['seed', [-7.04e+18, 6.82e+18]],
 ['ntrees', [14.0, 50.0]],
 ['distributionstopping_metric', [nan, nan]],
 ['stopping_tolerance', [0.008312003, 0.008312003]]]

Tried to do normal Grid Search to find important hyperparameter in ntrees but it did not work 

In [196]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'ntrees' : [32, 46, 14, 18, 50]

}
# Create a based model
etr = ExtraTreesRegressor()
# Instantiate the grid search model
reg = GridSearchCV(estimator = etr, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

__DRF__

In [25]:
#importing the file
df = h2o.import_file(path="DRF.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [26]:
df.head()

drf_id,name,nfolds,keep_cross_validation_models,keep_cross_validation_predictions,fold_assignment,stopping_metric,stopping_tolerance,seed,distribution
1,DRF_1_AutoML_20190418_132524,5,FALSE,TRUE,Modulo,AUTO,0.008312,-6.25e+18,gaussian
2,DRF_1_AutoML_20190418_135050,5,FALSE,TRUE,Modulo,deviance,0.008312,-4.44e+18,gaussian
3,DRF_1_AutoML_20190418_140346,5,FALSE,TRUE,Modulo,deviance,0.008312,2.36e+18,gaussian
4,DRF_1_AutoML_20190418_142243,5,FALSE,TRUE,Modulo,deviance,0.008312,-1.13e+18,gaussian
5,DRF_1_AutoML_20190418_144229,5,FALSE,TRUE,Modulo,deviance,0.008312,3.86e+18,gaussian
6,DRF_1_AutoML_20190418_150256,5,FALSE,TRUE,Modulo,deviance,0.008312,8.56e+18,gaussian


__2 Trying to find the range of hyperparameters__

In [27]:
df.types

{'drf_id': 'int',
 'name': 'string',
 'nfolds': 'int',
 'keep_cross_validation_models': 'enum',
 'keep_cross_validation_predictions': 'enum',
 'fold_assignment': 'enum',
 'stopping_metric': 'enum',
 'stopping_tolerance': 'real',
 'seed': 'int',
 'distribution': 'enum'}

Here, we are finding the range of all the column which are numerical and added to a list called ranges.

In [28]:
ranges = []
for key,val in df.types.items():
    print(key,val)
    if val == 'int': 
         ranges.append([key,[df[key].min(),df[key].max()]])
    if val == 'real':
         ranges.append([key,[df[key].min(),df[key].max()]])

drf_id int
name string
nfolds int
keep_cross_validation_models enum
keep_cross_validation_predictions enum
fold_assignment enum
stopping_metric enum
stopping_tolerance real
seed int
distribution enum


In [29]:
ranges

[['drf_id', [1.0, 6.0]],
 ['nfolds', [5.0, 5.0]],
 ['stopping_tolerance', [0.008312003, 0.008312003]],
 ['seed', [-6.25e+18, 8.56e+18]]]

Parameters which are supported by grid search for DRF are mtries and categorical_encoding are not present in DRF file to perform GridSearch 

__3 Compare the range of values across the models for different hyperparameters__ :

1 nfolds 

nfolds value is same for all the algorithms  

2 seed 

seed value ranged from -9e+18 to 8.93e+18 across the models for all the algorithmns

3 stopping_tolerance

It is present is all the algorithmns except GLM and its value is 0.008312003 for all models

4 ntrees

ntress are the hyperparameters for only XGBoost and XRT and the range for XGBoost is from 13 to 276 while for XRT is form 15 to 50 

XGboost and GBM has some same hyperparameters such as max_depth, min_rows, sample_rate

- max_depth ranges for both algorithms are almost similar XGBoost - (5,20) & GBM - (3,16)

- min_rows - XGboost values varied from 0.01 to 20 while for GBM its from 3 to 16  

- sample_rate - both algorithmns are same sample_rate of 0.5 to 1 



